In [4]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
import openai

In [5]:
# Load Data
df = pd.read_csv('data.csv')

In [6]:
# HELPER METHODS
def parse_legislation(text):
    laws = []
    if pd.isna(text):
        return laws
    for entry in re.split(r"\n|;", text):
        parts = entry.split(":", 1)
        if len(parts) == 2:
            name = parts[0].strip()
            desc = parts[1].strip()
            laws.append({'name': name, 'description': desc})
    return laws

In [7]:
df['Legislation_Parsed'] = df['legislation'].apply(parse_legislation)
df.to_csv('data2.csv')

In [8]:
feature_cols = [
    'GII_Rank', 'Military_Expenditure',
    'Proportion of women subjected to physical and/or sexual violence in the last 12 months (% of ever partnered women ages 15-49)',
    'Mean age at first marriage, female',
    'Women making their own informed decisions regarding sexual relations, contraceptive use and reproductive health care  (% of women age 15-49)',
    'There is legislation on sexual harassment in employment (1=yes; 0=no)',
    'Women and men have equal ownership rights to immovable property (1=yes; 0=no)',
    'CEDAW', 'Arms_Trade_Treaty'
]

In [9]:
def compare_country_to_peers(df, country, feature_cols):
    base = df[df['Country'] == country].iloc[0]
    cluster_peers = df[(df['Cluster'] == base['Cluster']) & (df['Country'] != country)]
    region_peers = df[(df['Region'] == base['Region']) & (df['Country'] != country)]
    metrics = {}
    for col in feature_cols:
        # Convert the column to numeric, coercing errors to NaN
        cluster_peers[col] = pd.to_numeric(cluster_peers[col], errors='coerce')
        region_peers[col] = pd.to_numeric(region_peers[col], errors='coerce')

        val = base[col]
        cluster_avg = cluster_peers[col].mean()
        region_avg = region_peers[col].mean()
        metrics[col] = {
            'value': val,
            'cluster_avg': cluster_avg,
            'region_avg': region_avg
        }
    return metrics

In [10]:
stories = ['''Judicial hiring practices reveal entrenched gendered and ideological biases. Male and conservative judges disproportionately hire male clerks, often perceiving women as less ideologically aligned, particularly in conservative circles. This dynamic is exacerbated by the decentralized hiring process, where judges receive hundreds of applications per position and often rely on subjective criteria. For example, a 2017 study found that women clerks were more likely to be hired by liberal judges, who prioritize descriptive representation, whereas conservative judges focused on ideological "fit." These practices perpetuate a cycle where women face barriers to entering influential legal roles, indirectly shaping the pipeline for future judges and policymakers. The lack of diversity among clerks not only skews judicial decisions but also reinforces systemic inequalities in the legal profession''',
                       '''My name is Diana, I am 44 years old and I live in Mid -Jutland. I work at the Danish Tax Agency (Skattestyrelsen). I have two children, one is 13, he turned 13 on Friday and one that is 6. I am alone. I think it has been hard. We have been sent home a lot. I have been home, all in all, for 8 months. It has been hard. At one time the big one was also home from school and Michelle was home in the beginning of the corona time. So I had Michelle home from kindergarten, she was 4 and then I had Christi an home from school and then I had work. So I felt that I was max pressured in terms of being enough in all areas. If it wasn't Michelle pulling me in the one arm, it was Christian having trouble getting on his school stuff and figuring out what he had to do or shouldn't do. And then there was work where there were expectations. I think it has been hard. At one point I completely lost motivation. I sort of felt that nothing mattered, because I wasn't doing enough anywhere. I felt I was running around in my own bubble. The only contact I had was with my sister, my father and my brother. At that time I was starting to think that I didn't want to continue, now it was simply too much. I might as well stop, because I was not enough anywhere. I thought it was hard . It affected my mood, my energy and the general feeling of being able to do things. I didn't feel like I had energy to do anything. It was just a nightmare. I contacted my manager, because I have been down with stress before and it was those kind of sympt oms that I was starting to feel again. They were returning. I felt that no matter what I did I was insufficient everywhere. I was allowed to go to the office even though we were sent home. It was approved by the deputy director, because if not I wouldn't have been allowed to go in. I was allowed to go in together with another colleague, so there was something outside of these four walls, where I just sat staring into the air. That made a difference. It helped. It was hard. The teams meetings and everything we had during the time it didn't give me any thing. Even if I could sit and talk with my colleagues, it didn't give me what it gave me to get away from home. Sitting at home constantly took my peep away. It was strenuous . Just getting out and being able to sit at my office, it made a huge difference. We had to show the good example, because we are a public agency. I felt that I was being punished. If the others could go physically to their office, because it's just a reco mmendation, then why couldn't we. That was hard for me to accept. We are actually also people, even though we work in public affairs. One time I had to throw away everything that I was working on to help my son with his schoolwork. My own work had t o be put aside again. The teacher wasn't replying when my son asked. I had to contact the school and tell them that if teachers were supposed to be available they had to be there. I sort of had all of the roles. I was supposed to be the teacher, I was supp osed to be the mom, I was supposed to be the pedagogue and I was supposed to be an employee at the same time. I thought it was hard juggling these roles. In one way I had to be the loving caregiver, in another way I had to be the person to say that they ha d to sit quietly and at the same time I had to be the creative one giving my daughter something to do that she could sit with on her own. And then I had to work at the same time. I think it has been hard. I'm happy it's all over soon. When we were working online, no one could put their camera on, because it would make the connection too slow or cause problems in other ways. There was no one where I could say: you have to take over here, they don't want to do anything and they are not listening. It became a vicious circle and then I became this angry person. No matter what they said or did I was about to rip their heads off in the end. I thought: now they just had to stop, it couldn't be right. I also had to pull the break and say I don't know what to do. I r eached out to my dad and said ''do something, I don't have anymore energy, I don't know how to act, I just get angry with them all the time''. It wasn't very fun. It wasn't the type of mom that I wanted to be. With my employer I felt, I will do what I have t he time to do, but with my children I felt insufficient. I felt that no matter which strings I tried to pull, I didn't really feel sufficient. I had no more tools to use. In particular I remember one episode, where my boy came and told me: ''Mom, you are fr eaking angry all the time. You are just scolding''. That was the last drop. I then thought that I didn't want this. I'm not interested in being the mom that is just scolding, as soon as they open their mouths. Both me and my children are vaccinated. All thr ee of us have had corona at our home. My son had cold symptoms, my daughter didn't have any symptoms and I was really sick for 14 days, with coughing, shortage of breath and a sore throat. I didn't expect that I would have been hit so hard, when I recently had taken the third vaccine. Why did I have to take it, when I was hit so hard? I still cough and get a shortage of breath, when I walk up the stairs. Whenever my pulse increases, I cannot breathe.'''
                 ]

In [11]:
def build_formatted_prompt(country, row, df, similar_df, comparison_metrics, alternate, focus_areas):
    NAP_full = df.loc[df["Country"] == country, "NAP_Text_Clean"].iloc[0]
    laws = row['Legislation_Parsed']
    law_text = "\n".join([f"- {law['name']}: {law['description']}" for law in laws])

    similar_context = ""
    for _, r in similar_df.iterrows():
        if r['Legislation_Parsed']:
            for l in r['Legislation_Parsed']:
                similar_context += f"Referencing {l['name']} from {r['Country']}: {l['description']}.\n"

    comparison_text = ""
    for metric, vals in comparison_metrics.items():
        comparison_text += f"{metric}: {vals['value']} (Cluster avg: {vals['cluster_avg']:.2f}, Region avg: {vals['region_avg']:.2f})\n"

    # Check if NAP_Summary is present
    if alternate:
        # Use alternate prompt if NAP_Summary exists
        # ***PASTE ALTERNATE PROMPT HERE***
        full_prompt = f"""
        **You are a policy assistant tasked with refining and strengthening a country’s existing National Action Plan (NAP) for gender equality.
        **IN YOUR RESPONSE, DESIGNATE SECTION HEADERS WITH DOUBLE ASTERISKS (**) AND SUB_HEADERS (Each section of the NAP Outline) WITH SINGLE ASTERISKS (*).**
        You will be given the following information:**

        1. Country
        2. Existing legislation implemented by that country for women's rights
        3. Legislation from other countries to be used in analysis
        4. Which cluster each country belongs to. Cluster 0 should be disregarded as having any significance when drawing connections/conclusions in your response, only consider clustering if it is not cluster 0.
        5. Important data that should be used as points for comparison against other countries, especially countries in the same cluster or geographical region:
          - Proportion of women subjected to physical and/or sexual violence in the last 12 months (% ever-partnered, 15–49)
          – Sexual-harassment-in-employment legislation (1=yes; 0=no)
          – Equal immovable-property-ownership rights (1=yes; 0=no)
          – Women making their own informed decisions regarding sexual relations, contraceptive use and reproductive health care (% of women age 15–49)
          – Mean age at first marriage (female)
          – NAP_full (The existing NAP of the country)
          – CEDAW ratification status
          – Gender Inequality Index rank
          – Arms Trade Treaty signature status
          – Military Expenditure (if value has an 'M', the number is in millions of dollars. if value has a 'B', the figure is in billions of dollars.)
        6. 3 personal narratives and stories about experiences with gender inequality, discrimination, and identity

        7. The existing NAP of the current country at hand, with the following sections:
        – Actors
        – Timeframe
        – Objectives
        – Actions/Activities
        – Indicators
        – M&E (Monitoring and Evaluation)
        – Budget
        – Disarmament

        8. Focus areas for the refined NAP to address and specifically hone in on.

        Country:
        {country}

        Legislation:
        {law_text}

        Peer Legislation Examples:
        {similar_context}

        Data Comparison to Regional and Cluster Peers:
        {comparison_text}

        **Current NAP Summary:**
        {NAP_full}

        **Focus Areas:**
        {focus_areas}

        **Personal Narratives:**
        {", ".join(stories)}

        ---

        **1. Contextual & Comparative Analysis**
          - briefly introduce which focus areas are going to be addressed and how the revision will have a direct impact on those areas.
          - Compare each key statistic against at least two neighbouring or socio-politically similar countries, using actual figures that correspond factually to the csv file.
            - 'Peer' countries should be determined if one or more of the following are similar, based on your own knowledge and the data provided in the csv file:
              - regional proximity, economy, government structure, political climate, social/gender norms
          - Draw 2–3 insights on how legislative gaps constrain women’s rights, economic participation, or safety.

        **2. Legislative & Policy Gap Assessment**

          - Take silent note of any “0” flags in the legislation columns, and use this fact that weave rhetoric about the absence of legislation.
          - For each missing law, cite a peer country that has it—name the law verbatim as in your data (e.g. Advertising Amendment Act: Laws > Violence against women > Legislation translates to 'The Advertising Amendment Act', which is a law that addresses violence against women, especially in the workplace and industry...'') and summarize its impact on reducing gender-based violence or improving equality.
          - the primary focus should be on the focus areas that were previously outlined.
          - Link gaps in the current NAP to these missing legal provisions and to poorer outcomes in your comparative analysis.

        **3. Improvement Plan for the Existing NAP**

          - Highlight 2–3 Missing or Weak Issues in the current summary (e.g., absence of gender-responsive budgeting, lack of disarmament-related measures, insufficient youth-focused indicators).
          - Propose Concrete Amendments or New Measures to each (e.g., add a Gender Budgeting Unit within the Ministry of Finance; introduce a “Small Arms Risk Assessment” clause under disarmament).
          - Re-align Indicators for Clarity & Feasibility—rewrite each indicator as a SMART target (Specific, Measurable, Achievable, Relevant, Time-bound) and explain why this makes monitoring stronger.
          - again, focus on the focus areas outlined at the beginning.

        **4. Peer-Practice Recommendations**

          - For every major amendment, reference a “best practice” peer country with similar socioeconomic and political context.
          - Explain how that measure improved their relevant statistic

        **5. GII Ranking & Resource Realignment**

          - The Gender inequality index country-ranking is also provided, and is based on several factors:
          - maternal mortality, which may indicate a poor prioritization of women's healthcare
          - adolescent birth rate, which indicates early childbirth (age 15-19)
          - female seats in parliament
          - percent of females who have secondary education
          - female labor force participation
          - the lower the number, the better the country is for women. The lower the value in the GII column, the poorer the country's state of gender equality is.

          - Weave in the Gender Inequality Index rank to argue for prioritizing certain objectives (e.g., low female parliamentary representation calls for candidate quotas).
          - Identify any budgetary trade-offs—only if warranted—such as reallocating a small fraction of military expenditure to gender-sensitive projects, but focus on necessary new funding sources (e.g., international grants, public-private partnerships).

        **6. Incorporating Personal Experiences**

        – Weave the individual stories provided to you throughout the NAP and incoporate these in your response.
        – Use these narratives to contextualize each objective, action, and indicator—show how policies would directly address the barriers described.
        – Cite specific elements of the stories (e.g., access hurdles, cultural norms, support networks) when defining targets, timelines, and budgets to ensure measures remain grounded in lived realities.
        – Reference personal testimonies in monitoring frameworks, explaining how regular collection of such accounts will track progress and highlight unforeseen gaps.
        – Throughout the NAP, integrate at least two representative narratives in each major section to keep policy recommendations connected to the people they serve.
        - The prompt should not be expliclty referencing the narratives, specific peoples' names, etc. Instead, it should be subtly incorporating experiences discussed in these narratives for this country to improve legislative activism.

        **7. Revised NAP Outline**

          The revision should have a clear focus on the specific focus areas previously listed. It should be fairly obvious which focus areas are being addressed.

          **first structure a streamlined skeleton of the full NAP based on the existing summary, but with your amendments integrated under each section:**

          - Updated Objectives (3–5, incorporating new measures)
          - Time-bound Actions & Responsible Actors
          - SMART Indicators & M&E Plan (with brief rationale sentences)
          - Tentative Budget & Funding Sources (realistic figures, or an amount aligned with comparable countries)
          - Disarmament/Gender Security Measures
          - NEVER use variables or placeholders. Saying something vague like $X for a budget is unnacceptable. Either use concrete figures or don't use them at all.

          - New Legislative Proposals (name, scope, enforcement mechanism)

          **REMEMBER, THE FOCUS OF THE NAP SHOULD BE MANIFESTLY ADDRESSING THE ISSUES THAT FALL UNDER THE TOPICS OUTLINED IN THE FOCUS AREAS AT THE BEGINNING OF THIS INSTRUCTION.**
          **Use this skeleton to generate a final output that resembles a complete NAP. Essentially, the best of both worlds from your response's generated suggestions and the already-present national action plans. IT IS OK TO NOT HAVE CHANGES FOR CERTAIN ASPECTS: you can keep things the same as outlined in the summary if there doesn't need to be immediate change**


          YOUR RESPONSE SHOULD BE POLISHED AND PROFESSIONAL, AND SHOULD NOT APPEAR SIMPLY AS A 'RESPONSE' BUT AS A PROFESSIONAL POLICY MEMO THAT DOES NOT REFERENCE INSTRUCTION OR THOUGHTS DURING THE RESPONSE.

        **8. Final Commentary on Changes**

          - At the end, briefly list all which issues were added or strengthened, how indicators were re-defined, and why these changes will make the NAP more effective, concrete, and practicable.

        **9. Referenced Countries & Cited Legislation**

          **At the very bottom, include which laws and which countries were cited throughout the response. separate it (whitespace) from the rest of the response:**

          - Referenced Countries: [Peer A, Peer B, Peer C]
          - Cited Legislation: [Peer A: Law Title, Peer B: Law Title, …]

        """
    else:
        # Use original prompt if NAP_Summary is missing
        full_prompt = f"""
        **You are a policy assistant that drafts a country’s National Action Plan (NAP) for gender equality.
        **IN YOUR RESPONSE, DESIGNATE SECTION HEADERS WITH DOUBLE ASTERISKS (**) AND SUB_HEADERS (Each section of the NAP Outline) WITH SINGLE ASTERISKS (*).**
        You will be given the following information:**

        1. Country
        2. Existing legislation implemented by that country for women's rights
        3. Legislation from other countries to be used in analysis
        4. Which cluster each country belongs to. Cluster 0 should be disregarded as having any significance when drawing connections/conclusions in your response, only consider clustering if it is not cluster 0.
        5. Important data that should be used as points for comparison against other countries, especially countries in the same cluster or geographical region:
          - Proportion of women subjected to physical and/or sexual violence in the last 12 months (% ever-partnered, 15–49)
          – Sexual-harassment-in-employment legislation (1=yes; 0=no)
          – Equal immovable-property-ownership rights (1=yes; 0=no)
          – Women making their own informed decisions regarding sexual relations, contraceptive use and reproductive health care (% of women age 15–49)
          – Mean age at first marriage (female)
          – NAP_full (The existing NAP of the country)
          – CEDAW ratification status
          – Gender Inequality Index rank
          – Arms Trade Treaty signature status
          – Military Expenditure (if value has an 'M', the number is in millions of dollars. if value has a 'B', the figure is in billions of dollars.)

        6. Focus areas that your NAP will address

        **Country:**
        {country}

        **Legislation:**
        {law_text}

        **Peer Legislation Examples:**
        {similar_context}

        **Data Comparison to Regional and Cluster Peers:**
        {comparison_text}

        **Focus Areas:**
        {focus_areas}

        **Personal Narratives:**
        {", ".join(stories)}

        ---

        **1. Contextual Analysis**

          - Compare each statistic against at least two neighboring or socio‑politically similar countries (e.g., proximity, economy, democratic structure, defence profile).
          - Draw out 2–3 key insights (e.g. “Mean marriage age is x years younger than in [Peer], likely limiting women’s workforce entry because they are now bound to a husband.”).

        **2. Legislative Gap Assessment**

          - Identify any “0” flags in the legislation columns.
          - For each missing law, cite a peer country that has it and link it to better outcomes in your comparative analysis. Cite specific laws from the legislation column.
          - Cite the specific name of the law and a description of what that law addresses and which problems it attempts to fix. Make sure the name of the law is the title verbatim (example: ['Tunisia's Advertising Amendment Act, a law addressing violence against women'] appears as such on the csv: Advertising Amendment Act: Laws > Violence against women > Legislation)
          - Always use stats when possible: if talking about marriage age or violence rates, use the actual numbers and percentages at your disposal to make a definitive comparison.

        **3. Writing the NAP**

          - **Draft a complete NAP** with a clear focus on the specific focus areas previously listed. It should be fairly obvious which focus areas are being addressed. Your NAP should be structured as follows:
            - Clear Objectives (3–5)
            - Time‑bound Actions & Actors
            - Indicators & M&E plan
            - Tentative Budget & Funding sources
            - Disarmament/defence‑related gender measures
            - Required new legislation (with model text references)
            - Highlighting 2–3 missing issues (based on your gap analysis)
            - Proposing concrete amendments or new measures
            - Re‑aligning indicators for clarity and feasibility
            - Discuss poor budgetary and funding decisions that could potentially hinder women's progress (military expenditure is unecessarily high, resources can be diverted to a new department, etc.) This is an example, and diversion of resources from the military budget should not be referenced every time, only when deemed fit.

        **4. Peer‑Practice Recommendations**

          - For each major recommendation, reference a “best practice” peer.
          - Explain how adopting that measure improved a comparable statistic there.

        **5. GII Ranking Analysis**

          - The Gender inequality index country-ranking is also provided, and is based on several factors:
          - maternal mortality, which may indicate a poor prioritization of women's healthcare
          - adolescent birth rate, which indicates early childbirth (age 15-19)
          - female seats in parliament
          - percent of females who have secondary education
          - female labor force participation
          - the lower the number, the better the country is for women. The lower the value in the GII column, the poorer the country's state of gender equality is.
          - Use this score to weave a narrative and suggest changes to decrease bias etc.

        **6. Incorporating Personal Experiences**

          – Weave the individual stories provided to you throughout the NAP and incoporate these in your response.
          – Use these narratives to contextualize each objective, action, and indicator—show how policies would directly address the barriers described.
          – Cite specific elements of the stories (e.g., access hurdles, cultural norms, support networks) when defining targets, timelines, and budgets to ensure measures remain grounded in lived realities.
          – Reference personal testimonies in monitoring frameworks, explaining how regular collection of such accounts will track progress and highlight unforeseen gaps.
          – Throughout the NAP, integrate at least two representative narratives in each major section to keep policy recommendations connected to the people they serve.
          - The prompt should not be expliclty referencing the narratives, specific peoples' names, etc. Instead, it should be subtly incorporating experiences discussed in these narratives for this country to improve legislative activism.


        **7. Line of Reasoning**
          - There should be a clear line of reasoning that fully fleshes out ideas and logic.
          - *Example*
              - The Arms Trade Treaty (ATT) is the first legally binding international agreement to explicitly recognize and require states to assess and mitigate the risk of gender-based violence (GBV) linked to arms transfers.
              - A country’s absence from the list of ATT signatories suggests a lower commitment to these gender-related provisions and, by extension, may correlate with weaker political will to address broader gender equality issues.
              - By not signing the ATT, a government effectively opts out of legally bound gender‑sensitive arms assessments. This can signal a deprioritization of preventive measures against the use of arms in perpetrating GBV.
              - *If a country has not signed the ATT, make sure the points above are made in the final output*
          - The writing should take on a professional tone with semi-neutral but factual rhetoric.
          - ALWAYS be concrete, avoid vague statements. Always addresss how or why something is happening, and how to address them. Statements that simply state 'if it was fixed, it would be better' should not be used. Instead, articulate HOW it can be fixed and the direct steps that need to be taken.
          - When stating the Indicators & M&E, there should be complete sentences explaining what each one is and why each of the measures are important, not simply bullet-listing the measures.
          - when discussing age of marriage etc, REFERENCE THE ACTUAL AGE. Always use available statistics to make a definitive comparison; the numbers are all provided in the csv. You must reference percentages, ages, rates, numbers, amounts, etc. STATS MUST BE USED.
          - NEVER use variables or placeholders. Saying something vague like $X for a budget is unnacceptable. Either use concrete figures or don't use them at all.

        8. **Final Output**
        - Write as a polished policy memo, with sections:
          1. Executive Summary, including which focus areas will be discussed in the NAP
          2. Context & Comparative Analysis
          3. Legislative Gaps & Peer Practices
          4. NAP (New or Revised)
          5. Implementation Roadmap (Actors, Timeline, Budget, M&E) ALL OF THESE MUST BE REALISTIC FIGURES AND STATEMENTS: for example, the budget has to be a reasonable number and a reasonable percentage of the GDP, while also not being too small.
            - It should be based on previously known budgets as well as economic restraints, but for example, $500,000 is far too little, but $10 billion is too much. This logic should be applied concretely to everything: Actors, Timeline, Budget, M&E. Feasible but not ineffective/underwhelming
        - **REMEMBER, THE FOCUS OF THE NAP SHOULD BE MANIFESTLY ADDRESSING THE ISSUES THAT FALL UNDER THE TOPICS OUTLINED IN THE FOCUS AREAS AT THE BEGINNING OF THIS INSTRUCTION.**
        - YOUR FINAL RESPONSE SHOULD HAVE ALL OF THE INFORMATION OUTLINED IN STEPS 1-6, BUT THE HEADERS MUST READ AS SPECIFIED HERE IN STEP 7.

          **At the very bottom, include the following simply for the developers to reference. separate it from the rest of the response:**

          - Referenced Countries: [Peer A, Peer B, Peer C]
          - Cited Legislation: [Peer A: Law Title, Peer B: Law Title, …]

        """
    print("*********************Using prompt:**********************")
    print(full_prompt)
    return full_prompt

In [13]:
def generate_nap_gpt(prompt, max_tokens=1024):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": "You are a helpful policy drafting assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0.7
    )
    # Access the content using .content instead of ['content']
    return response.choices[0].message.content

In [14]:
country = "Afghanistan"

print(f"\n▶ Generating NAP for: {country}")

row = df[df['Country'] == country].iloc[0]
similar = df[(df['Cluster'] == row['Cluster']) & (df['Country'] != country)].head(3)
comparison = compare_country_to_peers(df, country, feature_cols)
prompt = build_formatted_prompt(country, row, df, similar, comparison, alternate=False, focus_areas=["Political Representation", "Education"])
nap_output = generate_nap_gpt(prompt)

print(f"\n✅ Generated NAP for {country}:\n{nap_output}")


▶ Generating NAP for: Afghanistan
*********************Using prompt:**********************

        **You are a policy assistant that drafts a country’s National Action Plan (NAP) for gender equality. 
        **IN YOUR RESPONSE, DESIGNATE SECTION HEADERS WITH DOUBLE ASTERISKS (**) AND SUB_HEADERS (Each section of the NAP Outline) WITH SINGLE ASTERISKS (*).**
        You will be given the following information:**

        1. Country
        2. Existing legislation implemented by that country for women's rights
        3. Legislation from other countries to be used in analysis
        4. Which cluster each country belongs to. Cluster 0 should be disregarded as having any significance when drawing connections/conclusions in your response, only consider clustering if it is not cluster 0.
        5. Important data that should be used as points for comparison against other countries, especially countries in the same cluster or geographical region:
          - Proportion of women subjected 

<ipython-input-9-0958fdaa1fc8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_peers[col] = pd.to_numeric(cluster_peers[col], errors='coerce')
<ipython-input-9-0958fdaa1fc8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_peers[col] = pd.to_numeric(region_peers[col], errors='coerce')



✅ Generated NAP for Afghanistan:
**Context & Comparative Analysis**

*Statistical Comparison with Neighboring Countries:*

Afghanistan has a relatively high proportion of women subjected to physical and/or sexual violence in the last 12 months (35.0%). In comparison to neighboring countries, this rate is significantly higher than Cluster and Region averages. For example, when compared to [Peer A], where the percentage is much lower at X%, it highlights the urgent need for targeted interventions in Afghanistan to address violence against women.

*Key Insights:*

1. The mean age at first marriage for females in Afghanistan (21.5) is lower than the Region average (22.48), potentially limiting women's opportunities for education and workforce participation. This contrasts with [Peer B], where the mean age is higher at X years, enabling women to engage more in societal and economic activities.

2. Legislation on sexual harassment in employment and equal immovable property ownership rights 